In [77]:
from time import sleep
from tqdm import tqdm

import numpy as np
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe
from google.oauth2.service_account import Credentials

from keys import GOOG_API_KEY

In [78]:
## Load dataframes
# Polling averages
pres = pd.read_csv('averages/presidential_gen_approval.csv')
pres_issue = pd.read_csv('averages/presidential_issue_approvals.csv')
pres_rv = pd.read_csv('averages/presidential_RV_approval.csv')
generic_ballot = pd.read_csv('averages/generic_ballot.csv')

# Polls display tables
pres_polls = pd.read_csv('transformed_tables/gen_approval_polls.csv')
issue_polls = pd.read_csv('transformed_tables/issue_approval_polls.csv')
gb_polls = pd.read_csv('transformed_tables/generic_ballot_polls_disp.csv')

In [79]:
# Define the scope
scopes = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive"
]

# Authenticate with the service account
credentials = Credentials.from_service_account_file(
    GOOG_API_KEY,
    scopes=scopes
)

# Authorize gspread
client = gspread.authorize(credentials)

## Polling Averages

In [80]:
spreadsheet = client.open("snoutcounter-data")
worksheet = spreadsheet.worksheet("approval-stats")
worksheet.clear()

set_with_dataframe(worksheet, pres.reset_index())

In [81]:
worksheet = spreadsheet.worksheet("approval_stats_RV")
worksheet.clear()

set_with_dataframe(worksheet, pres_rv.reset_index())

In [82]:
issues = ['economy', 'immigration', 'inflation', 'trade_tariffs', 'foreign_policy', 'healthcare', 'crime']
issue_suffixes = ['econ', 'imm', 'infl', 'trade', 'fp', 'healthcare', 'crime']

for iss, suf in tqdm(zip(issues, issue_suffixes)):
    worksheet = spreadsheet.worksheet(f"approval_stats_{suf}")
    worksheet.clear()

    issue_df = pres_issue[pres_issue['issue'] == iss]
    set_with_dataframe(worksheet, issue_df.reset_index())

7it [00:05,  1.24it/s]


In [83]:
## Net issue approval
net_issues = pd.pivot_table(pres_issue[pres_issue['issue'].isin(issues)], 
                            values=['net', 'net_lower_ci', 'net_upper_ci'], index=['end_date'], columns=['issue'],
                           aggfunc="first").reset_index()
net_issues.columns = ['_'.join(col).strip() for col in net_issues.columns.values]
net_issues.head()

,end_date_,net_crime,net_economy,net_foreign_policy,net_healthcare,net_immigration,net_inflation,net_trade_tariffs,net_lower_ci_crime,net_lower_ci_economy,...,net_lower_ci_immigration,net_lower_ci_inflation,net_lower_ci_trade_tariffs,net_upper_ci_crime,net_upper_ci_economy,net_upper_ci_foreign_policy,net_upper_ci_healthcare,net_upper_ci_immigration,net_upper_ci_inflation,net_upper_ci_trade_tariffs
0,2025-01-21,0.0,7.849186,3.849186,0.000000,9.849186,-4.150814,-0.150814,0.0,7.849186,...,9.849186,-4.150814,-0.150814,0.0,7.849186,3.849186,0.000000,9.849186,-4.150814,-0.150814
1,2025-01-22,0.0,7.849186,3.849186,0.000000,9.849186,-4.150814,-0.150814,0.0,7.849186,...,9.849186,-4.150814,-0.150814,0.0,7.849186,3.849186,0.000000,9.849186,-4.150814,-0.150814
2,2025-01-23,0.0,3.158170,3.849186,-13.153535,3.818400,-4.150814,-0.150814,0.0,-3.296228,...,-4.479394,-4.150814,-0.150814,0.0,9.612567,3.849186,-13.153535,12.116194,-4.150814,-0.150814
3,2025-01-24,0.0,3.158170,3.849186,-13.153535,3.818400,-4.150814,-0.150814,0.0,-3.296228,...,-4.479394,-4.150814,-0.150814,0.0,9.612567,3.849186,-13.153535,12.116194,-4.150814,-0.150814
4,2025-01-25,0.0,3.158170,3.849186,-13.153535,3.818400,-4.150814,-0.150814,0.0,-3.296228,...,-4.479394,-4.150814,-0.150814,0.0,9.612567,3.849186,-13.153535,12.116194,-4.150814,-0.150814


In [84]:
worksheet = spreadsheet.worksheet("net_approval_issues")
worksheet.clear()

set_with_dataframe(worksheet, net_issues.reset_index())

In [85]:
worksheet = spreadsheet.worksheet("generic_ballot_stats")
worksheet.clear()

set_with_dataframe(worksheet, generic_ballot.reset_index())

## Polls Display Tables

In [86]:
pres_polls['sponsors'] = '^Sponsor: ' + pres_polls['sponsors'] + '^'
pres_polls['sponsors'] = pres_polls['sponsors'].fillna('')
pres_polls = pres_polls.sort_values(['total_weight'], ascending=False)
pres_polls = pres_polls.rename({
    'pollster':'Pollster', 'approve':'Approve', 'disapprove':'Disapprove', 'net':'Net', 
    'total_weight':'Weight', 'start_date':'Start Date', 'end_date':'End Date', 'url':'URL', 'adj_net':'Adjusted Net'}, axis='columns')
pres_polls['Sample'] = pres_polls['sample_size'].astype(int).astype(str) + ' ' + pres_polls['population']
pres_polls['Start Date'] = pd.to_datetime(pres_polls['Start Date']).astype(str)
pres_polls['End Date'] = pres_polls['End Date'].astype(str)
pres_polls = pres_polls.reset_index()
pres_polls['Pollster'] = ('<a href="' + pres_polls['URL'] + '" style="color:black; text-decoration:underline;"><u>' + 
                          pres_polls['Pollster'] + '</u></a>' + pres_polls['sponsors'])
pres_polls = pres_polls[['Pollster', 'Start Date', 'End Date', 'Sample', 'Weight', 'Approve', 'Disapprove', 'Net', 'Adjusted Net']]
pres_polls

,Pollster,Start Date,End Date,Sample,Weight,Approve,Disapprove,Net,Adjusted Net
0,"<a href=""https://napolitannews.org/jobapproval...",2026-01-07,2026-01-14,3000 RV,1.463914e-01,48.0,51.0,-3.0,-12.921862
1,"<a href=""https://maristpoll.marist.edu/polls/u...",2026-01-12,2026-01-13,1408 A,1.113055e-01,38.0,56.0,-18.0,-15.694776
2,"<a href=""https://www.yahoo.com/news/us/article...",2026-01-08,2026-01-12,1709 A,9.212593e-02,40.0,56.0,-16.0,-14.279151
3,"<a href=""https://poll.qu.edu/poll-release?rele...",2026-01-08,2026-01-12,1133 RV,8.438267e-02,40.0,54.0,-14.0,-10.016768
4,"<a href=""https://www.cbsnews.com/news/u-s-acti...",2026-01-05,2026-01-07,2325 A,6.210508e-02,41.0,59.0,-18.0,-16.279151
...,...,...,...,...,...,...,...,...,...
525,"<a href=""https://pro.morningconsult.com/tracke...",2025-01-24,2025-01-26,2302 RV,1.775566e-19,52.0,44.0,8.0,3.825468
526,"<a href=""https://substack.com/inbox/post/15539...",2025-01-21,2025-01-21,742 A,1.673661e-19,49.0,36.0,13.0,8.218111
527,"<a href=""https://quantusinsights.org/f/februar...",2025-02-01,2025-02-03,1000 RV,1.569060e-19,51.9,45.3,6.6,-0.015417
528,"<a href=""https://www.ipsos.com/en-us/reuters-i...",2025-01-20,2025-01-21,1077 A,1.436730e-19,47.0,41.0,6.0,10.012954


In [87]:
worksheet = spreadsheet.worksheet("polls_display_table")
worksheet.clear()

set_with_dataframe(worksheet, pres_polls)

In [88]:
issue_polls['sponsor'] = '^Sponsor: ' + issue_polls['sponsor'] + '^'
issue_polls['sponsor'] = issue_polls['sponsor'].fillna('')
issue_polls = issue_polls.sort_values(['issue', 'total_weight'], ascending=False)
issue_polls = issue_polls.rename({
    'pollster':'Pollster', 'approve':'Approve', 'disapprove':'Disapprove', 'net':'Net', 
    'total_weight':'Weight', 'start_date':'Start Date', 'end_date':'End Date', 'url':'URL', 'adj_net':'Adjusted Net',
    'issue':'Issue'}, axis='columns')
issue_polls['Sample'] = issue_polls['sample_size'].astype(int).astype(str) + ' ' + issue_polls['population']
issue_polls['Start Date'] = pd.to_datetime(issue_polls['Start Date']).astype(str)
issue_polls['End Date'] = issue_polls['End Date'].astype(str)
issue_polls = issue_polls.reset_index()
issue_polls['Pollster'] = ('<a href="' + issue_polls['URL'] + '" style="color:black; text-decoration:underline;"><u>' + 
                          issue_polls['Pollster'] + '</u></a>' + issue_polls['sponsor'])
issue_polls = issue_polls[issue_polls['Issue'].isin(issues)]
issue_polls['Issue'] = issue_polls['Issue'].replace({
    'economy': 'Economy',
    'immigration': 'Immigration',
    'foreign_policy': 'Foreign policy',
    'inflation': 'Inflation/prices',
    'trade_tariffs': 'Trade/tariffs',
    'healthcare': 'Healthcare',
    'crime': 'Crime'
})
issue_polls = issue_polls[['Pollster', 'Start Date', 'End Date', 'Issue', 'Sample', 'Weight', 'Approve', 'Disapprove', 'Net', 'Adjusted Net']]
issue_polls

,Pollster,Start Date,End Date,Issue,Sample,Weight,Approve,Disapprove,Net,Adjusted Net
49,"<a href=""https://pro.morningconsult.com/tracke...",2026-01-09,2026-01-12,Trade/tariffs,2201 RV,1.669119e-02,43.0,48.0,-5.0,-17.328894
50,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2026-01-09,2026-01-12,Trade/tariffs,1602 A,1.578531e-02,37.0,56.0,-19.0,-20.319552
51,"<a href=""https://s3.documentcloud.org/document...",2026-01-09,2026-01-12,Trade/tariffs,1209 A,1.251917e-02,37.0,62.0,-25.0,-20.056279
52,"<a href=""https://pro.morningconsult.com/tracke...",2026-01-02,2026-01-04,Trade/tariffs,2201 RV,7.375221e-03,45.0,44.0,1.0,-11.328894
53,"<a href=""https://www.atlasintel.org/poll/usa-n...",2025-12-15,2025-12-19,Trade/tariffs,2315 A,4.477267e-03,35.0,61.0,-26.0,-27.153535
...,...,...,...,...,...,...,...,...,...,...
1220,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2025-02-23,2025-02-25,Crime,1604 A,6.818464e-14,47.0,37.0,10.0,8.680448
1221,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2025-02-16,2025-02-18,Crime,1603 A,3.933175e-14,48.0,37.0,11.0,9.680448
1222,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2025-02-09,2025-02-11,Crime,1595 A,2.273783e-14,45.0,41.0,4.0,2.680448
1223,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2025-02-02,2025-02-04,Crime,1604 A,1.332409e-14,48.0,35.0,13.0,11.680448


In [89]:
worksheet = spreadsheet.worksheet("polls_display_issues")
worksheet.clear()

set_with_dataframe(worksheet, issue_polls)

In [90]:
gb_polls['sponsors'] = '^Sponsor: ' + gb_polls['sponsors'] + '^'
gb_polls['sponsors'] = gb_polls['sponsors'].fillna('')
gb_polls = gb_polls.sort_values(['total_weight'], ascending=False)
gb_polls = gb_polls.rename({
    'pollster':'Pollster', 'dem':'Democrats', 'rep':'Republicans', 'net':'Net', 
    'total_weight':'Weight', 'start_date':'Start Date', 'end_date':'End Date', 'url':'URL', 'adj_net':'Adjusted Net'}, axis='columns')
gb_polls['Sample'] = gb_polls['sample_size'].astype(int).astype(str) + ' ' + gb_polls['population']
gb_polls['Start Date'] = pd.to_datetime(gb_polls['Start Date']).astype(str)
gb_polls['End Date'] = gb_polls['End Date'].astype(str)
gb_polls = gb_polls.reset_index()
gb_polls['Pollster'] = ('<a href="' + gb_polls['URL'] + '" style="color:black; text-decoration:underline;"><u>' + 
                          gb_polls['Pollster'] + '</u></a>' + gb_polls['sponsors'])
gb_polls = gb_polls[['Pollster', 'Start Date', 'End Date', 'Sample', 'Weight', 'Democrats', 'Republicans', 'Net', 'Adjusted Net']]
gb_polls

,Pollster,Start Date,End Date,Sample,Weight,Democrats,Republicans,Net,Adjusted Net
0,"<a href=""https://www.yahoo.com/news/us/article...",2026-01-08,2026-01-12,1148 RV,1.366453e-01,45.0,40.0,-5.0,-5.006662
1,"<a href=""https://www.reuters.com/data/trumps-a...",2026-01-12,2026-01-13,941 RV,1.331177e-01,40.0,38.0,-2.0,-3.998906
2,"<a href=""https://www.cygn.al/polls/january-202...",2026-01-08,2026-01-08,1500 LV,1.329465e-01,48.1,44.6,-3.5,-4.494787
3,"<a href=""https://napolitannews.org/posts/gener...",2026-01-05,2026-01-08,2000 RV,1.120388e-01,47.0,46.0,-1.0,-4.513118
4,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2026-01-09,2026-01-12,1433 RV,8.814268e-02,44.0,40.0,-4.0,-4.006662
...,...,...,...,...,...,...,...,...,...
261,"<a href=""https://www.cygn.al/voters-back-kingm...",2025-01-09,2025-01-12,1500 LV,2.291589e-19,44.6,46.8,2.2,1.205213
262,"<a href=""https://mclaughlinonline.com/pols/wp-...",2025-01-22,2025-01-27,1000 LV,2.046899e-19,40.0,48.0,8.0,2.984673
263,"<a href=""https://quantusinsights.org/f/trump%E...",2025-01-22,2025-01-23,1000 RV,1.251235e-19,45.0,48.0,3.0,1.168192
264,"<a href=""https://www.cygn.al/poll-musk-most-ef...",2024-12-09,2024-12-11,1500 LV,6.627437e-21,44.6,48.4,3.8,2.805213


In [91]:
worksheet = spreadsheet.worksheet("generic_ballot_polls_display_table")
worksheet.clear()

set_with_dataframe(worksheet, gb_polls)